# Quick introduction

## Reading and plotting

In [ ]:
import pooch

test_data = "CTD-spiked-unfiltered.cnv.bz2"
url = f"https://github.com/pyoceans/python-ctd/raw/main/tests/data/{test_data}"

fname = pooch.retrieve(
    url=url,
    known_hash="sha256:1de4b7ce665d5cece925c5feb4552c13bbc19cef3e229bc87dfd77acb1a730d3",
)

In [ ]:
import ctd

cast = ctd.from_cnv(fname)
down, up = cast.split()

down.head()

It is a `pandas.DataFrame` with all the pandas methods and properties.

In [ ]:
type(cast)

But with extras for pre-processing and plotting a ocean vertical profiles.

In [ ]:
from matplotlib import style

style.use("seaborn-v0_8-whitegrid")

down["t090C"].plot_cast()
down["c0S/m"].plot_cast();

Sometimes it is useful to plot the second variable in a different axis so we can compare the two.

In [ ]:
ax0 = down["t090C"].plot_cast(label="Temperature (°C)")
ax1 = down["c0S/m"].plot_cast(
    ax=ax0,
    label="Conductivity (S/m)",
    color="orange",
    secondary_y=True,
)


ax0.grid(False)
ax1.grid(False)

ax0.legend(loc="lower left")
ax1.legend(loc="lower right");

`python-ctd` saves of the file metadata in a dictionary to make them easy to access later. 

In [ ]:
metadata = cast._metadata

metadata.keys()

In [ ]:
print(metadata["header"])

## Pre-processing steps

Usually the first pre-processing step is to filter the high frequency jitter in the pressure sensor with a low pass filter, here is a zoom in the pressure data (the `pandas` index) demonstrating it:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(down.index, label="unfiltered")
ax.plot(down.lp_filter().index, label="filtered")
ax.axis([20870, 20930, 557.5, 559])
fig.legend();

Thanks to [pandas_flavor](https://github.com/Zsailer/pandas_flavor) we can chain all the pre-processing steps together.

In [ ]:
down = down[["t090C", "c0S/m"]]

proc = (
    down.remove_above_water()
    .remove_up_to(idx=7)
    .despike(n1=2, n2=20, block=100)
    .lp_filter()
    .press_check()
    .interpolate()
    .bindata(delta=1, method="interpolate")
    .smooth(window_len=21, window="hanning")
)

proc.head()

## CTD derive

Now we can compute all the derived variables.

In [ ]:
lon, lat = metadata["lon"], metadata["lat"]
lon, lat

In [ ]:
import gsw

p = proc.index

SP = gsw.SP_from_C(proc["c0S/m"].to_numpy() * 10.0, proc["t090C"].to_numpy(), p)
SA = gsw.SA_from_SP(SP, p, lon, lat)
SR = gsw.SR_from_SP(SP)

CT = gsw.CT_from_t(SA, proc["t090C"].to_numpy(), p)
z = -gsw.z_from_p(p, lat)
sigma0_CT = gsw.sigma0(SA, CT)

proc = (
    proc.assign(SP=SP)
    .assign(SA=SA)
    .assign(SR=SR)
    .assign(CT=CT)
    .assign(z=z)
    .assign(sigma0_CT=sigma0_CT)
)

In [ ]:
labels = [
    r"Absolute Salinity (g kg$^{-1}$)",
    r"Reference Salinity (g kg$^{-1}$)",
    "Practical Salinity",
]


ax = proc[["SA", "SR", "SP"]].plot_cast(
    figsize=(5.25, 9),
    label=labels,
)
ax.set_ylabel("Pressure (dbar)")
ax.grid(True)
ax.legend()
ax.set_title("Salinities");

Last but not least let's tweak a three line plot with the main variables measured.

In [ ]:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)

In [ ]:
fig, ax0 = plt.subplots(figsize=(5, 9))

colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]

ax0.invert_yaxis()
ax1 = ax0.twiny()
ax2 = ax0.twiny()

(l0,) = ax0.plot(proc["CT"], proc.index, color=colors[0], label="CT")
ax0.set_xlabel("Conservative Temperature (°C)")

(l1,) = ax1.plot(proc["SA"], proc.index, color=colors[1], label="SA")
ax1.set_xlabel("Absolute Salinity (g kg$^{-1}$)")

(l2,) = ax2.plot(
    proc["sigma0_CT"],
    proc.index,
    color=colors[2],
    label=r"$\sigma_{0\_CT}$",
)
ax2.set_xlabel(r"$\sigma_{0\_CT}$ (kg m$^{-3}$)")

make_patch_spines_invisible(ax2)
ax2.spines["top"].set_position(("axes", 1.1))
ax2.spines["top"].set_visible(True)

ax0.xaxis.label.set_color(l0.get_color())
ax1.xaxis.label.set_color(l1.get_color())
ax2.xaxis.label.set_color(l2.get_color())

ax0.tick_params(axis="x", colors=l0.get_color())
ax1.tick_params(axis="x", colors=l1.get_color())
ax2.tick_params(axis="x", colors=l2.get_color())

ax0.grid(False)
ax1.grid(False)
ax2.grid(False)

ax0.legend(loc="lower left")
ax1.legend(loc="lower right")
ax2.legend(loc="lower center");